In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.unet_enhancement as unet
from malaya_speech.train.model import enhancement
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [5]:
x = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, 16384)
model = unet.Model(partitioned_x)

Tensor("strided_slice_42:0", shape=(?, 8192, 24), dtype=float32)
Tensor("strided_slice_43:0", shape=(?, 4096, 48), dtype=float32)
Tensor("strided_slice_44:0", shape=(?, 2048, 72), dtype=float32)
Tensor("strided_slice_45:0", shape=(?, 1024, 96), dtype=float32)
Tensor("strided_slice_46:0", shape=(?, 512, 120), dtype=float32)
Tensor("strided_slice_47:0", shape=(?, 256, 144), dtype=float32)
Tensor("strided_slice_48:0", shape=(?, 128, 168), dtype=float32)
Tensor("strided_slice_49:0", shape=(?, 64, 192), dtype=float32)
Tensor("strided_slice_50:0", shape=(?, 32, 216), dtype=float32)
Tensor("strided_slice_51:0", shape=(?, 16, 240), dtype=float32)
Tensor("strided_slice_52:0", shape=(?, 8, 264), dtype=float32)
Tensor("strided_slice_53:0", shape=(?, 4, 288), dtype=float32)
Tensor("sequential_38/leaky_re_lu_37/LeakyRelu:0", shape=(?, 4, 288), dtype=float32)
Tensor("sequential_39/leaky_re_lu_38/LeakyRelu:0", shape=(?, 8, 288), dtype=float32)
Tensor("sequential_40/leaky_re_lu_39/LeakyRelu:0", shape=

In [6]:
model.logits

<tf.Tensor 'sequential_51/activation_7/Tanh:0' shape=(?, 16384, 1) dtype=float32>

In [7]:
logits = tf.reshape(model.logits, (-1, 1))
logits = logits[:tf.shape(x)[0]]
logits

<tf.Tensor 'strided_slice_79:0' shape=(?, 1) dtype=float32>

In [8]:
snr = enhancement.loss.snr(logits, x)
sdr = enhancement.loss.sdr(logits, x)

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [10]:
import numpy as np

In [11]:
y, sr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 22050)

In [18]:
y_, snr_, sdr_ = sess.run([logits, snr, sdr], feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

(258867, 1)

In [13]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 22050)

In [14]:
snr_, sdr_

(-16.402222, 5.9038007e-07)

In [20]:
y

array([-0.00570545, -0.01051882, -0.01370371, ...,  0.0140948 ,
        0.01236383,  0.01233889])

In [21]:
y_[:,0]

array([0.42679474, 0.34322935, 0.5080514 , ..., 0.16915062, 0.18726589,
       0.22943994], dtype=float32)